# Mr Pi

## Notes
### Motor
Motor Function Recommended
Range 0-100 / reverse polarity to change direction 
#### Pins
- Motor 1 = 13 | Output on high while motor 2 on low rolls forward
- Motor 2 = 11 | Output on high while motor 1 on low rolls backwards
- Enable = 15  | Controls engine speed, PMW, 1KHz, Map Function Recommended

### Servo
**SERVO OPERATION RANGE 0-180 DEGREES**\
Servo Function Recommended
#### PINS
- Servo 1 = 12 | Controls servo position, PMW, 50Hz, Map Function Recommended

Set offset duty to 0.5\
set min/max duties - range (2.5-12.5) + offset

### UltraSonic Range
**OPERATION RANGE 3.5cm-220cm**\
Set Max Distance
Set timeout
Pulse Time Function & Sonar Functions Recommended
#### Pins
- Trigger = 16 | Emmits sonic pulse for .00001 secs
- Echo = 18    | Listens for sonic pulse

### Project goal
Use sonar signal as value for conditional checks\
If over send signal via pin 36 to pcf\
As distance approces 8cm reduce PWM to pin 36 
As distance approces 8cm rotate server either direction
Pin 36 power low if <= 5cm / backup / servo 90 Degrees / forward 2 sec / reset 
